# 📘 descricao_dados
### **Objetivo:**
- Descrever campos, tipos, exemplos e significado dos dados coletados; realizar EDA inicial e verificar qualidade para que o colega responda à seção “2. Compreensão dos Dados” do relatório.

## Setup

In [1]:
# Pacotes e paths
import json, os, io
from pathlib import Path
import pandas as pd
import numpy as np

# Raiz do repositório (ajuste se necessário)
ROOT = Path("..").resolve().parent if Path(".").name == "lab" else Path("..")
DATA = ROOT / "data" / "processed"
LAB_DATA = ROOT / "lab" / "data" / "processed"
QUALITY_DIR = ROOT / "data" / "quality"
QUALITY_DIR.mkdir(parents=True, exist_ok=True)

# Arquivos principais (existentes no seu projeto)
FILES = {
    "deputados": DATA / "deputados.json",
    "eventos_sessoes": DATA / "eventos_sessoes.csv",
    "freq_eventos": DATA / "freq_eventos.csv",
    "votos_deputados": DATA / "votos_deputados.csv",
    "ocupacoes": DATA / "ocupacoes.csv",
    "remuneracoes": DATA / "remuneracoes.csv",
    "cod_situacao_deputados": DATA / "cod_situacao_deputados.json",

    # opcionais / gerados no 01: (ficam no diretório lab/)
    "presencas": LAB_DATA / "presencas.csv",   # pode existir só em lab/data/processed
    "votacoes_consolidadas": DATA / "votacoes.csv",  # se tiver sido consolidado
}

def exists(p: Path) -> bool:
    try:
        return p.exists() and p.is_file()
    except Exception:
        return False


## Funções auxiliares para descrição e qualidade

In [2]:
def describe_dataframe(df: pd.DataFrame, sample_rows: int = 5):
    """Retorna (dtypes, nulos, amostra, cardinalidade) como DataFrames bonitos."""
    dtypes = df.dtypes.rename("dtype").to_frame()
    nulls = (df.isna().mean().round(4) * 100).rename("pct_null").to_frame()
    nunique = df.nunique(dropna=True).rename("nunique").to_frame()
    summary = dtypes.join(nulls, how="outer").join(nunique, how="outer").reset_index().rename(columns={"index":"column"})
    sample = df.head(sample_rows)
    return summary, sample

def check_pk_unique(df: pd.DataFrame, pk_cols):
    """Verifica unicidade de uma chave primária (lista de colunas)."""
    if not pk_cols: 
        return pd.DataFrame({"ok":[False], "msg":["pk_cols vazia"]})
    dup = df.duplicated(subset=pk_cols, keep=False)
    ok = not dup.any()
    msg = "OK (sem duplicatas)" if ok else f"Falha: {dup.sum()} duplicatas"
    return pd.DataFrame({"ok":[ok], "msg":[msg], "pk_cols":[pk_cols]})

def check_fk_in_pk(df_child: pd.DataFrame, child_cols, df_parent: pd.DataFrame, parent_cols):
    """Verifica integridade referencial: child[child_cols] ⊆ parent[parent_cols]."""
    child_key = df_child[child_cols].dropna()
    parent_key = df_parent[parent_cols].dropna()
    merged = child_key.merge(parent_key.drop_duplicates(), how="left", left_on=child_cols, right_on=parent_cols, indicator=True)
    missing = (merged["_merge"] == "left_only").sum()
    ok = missing == 0
    msg = "OK (todas as chaves filhas possuem pai)" if ok else f"Falha: {missing} chaves filhas sem pai"
    return pd.DataFrame({"ok":[ok], "msg":[msg], "child_cols":[child_cols], "parent_cols":[parent_cols], "missing":[missing]})

def safe_read_csv(path: Path, **kwargs):
    """Lê CSV se existir; senão retorna None."""
    if exists(path):
        try:
            return pd.read_csv(path, **kwargs)
        except Exception as e:
            print(f"[WARN] Falha ao ler CSV {path}: {e}")
    return None

def safe_read_json_records(path: Path):
    if exists(path):
        try:
            return pd.read_json(path, orient="records")
        except ValueError:
            # se for json convencional (lista de objetos), tenta via load
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            return pd.json_normalize(data)
        except Exception as e:
            print(f"[WARN] Falha ao ler JSON {path}: {e}")
    return None


## Catálogo dos datasets

In [3]:
catalog_rows = []
for name, path in FILES.items():
    if exists(path):
        if path.suffix.lower() == ".csv":
            df = safe_read_csv(path, nrows=1000)  # leitura parcial para não pesar
        elif path.suffix.lower() == ".json":
            df = safe_read_json_records(path)
        else:
            df = None
        shape = tuple(df.shape) if df is not None else (0,0)
        catalog_rows.append({
            "dataset": name,
            "path": str(path),
            "exists": True,
            "rows": shape[0],
            "cols": shape[1],
        })
    else:
        catalog_rows.append({
            "dataset": name,
            "path": str(path),
            "exists": False,
            "rows": 0,
            "cols": 0,
        })

catalog = pd.DataFrame(catalog_rows).sort_values(["exists","dataset"], ascending=[False, True]).reset_index(drop=True)
catalog


[WARN] Falha ao ler CSV ../data/processed/eventos_sessoes.csv: No columns to parse from file


,dataset,path,exists,rows,cols
0,cod_situacao_deputados,../data/processed/cod_situacao_deputados.json,True,8,4
1,deputados,../data/processed/deputados.json,True,513,9
2,eventos_sessoes,../data/processed/eventos_sessoes.csv,True,0,0
3,freq_eventos,../data/processed/freq_eventos.csv,True,513,2
4,ocupacoes,../data/processed/ocupacoes.csv,True,1000,4
5,presencas,../lab/data/processed/presencas.csv,True,1000,4
6,remuneracoes,../data/processed/remuneracoes.csv,True,513,4
7,votos_deputados,../data/processed/votos_deputados.csv,True,1000,4
8,votacoes_consolidadas,../data/processed/votacoes.csv,False,0,0


# 🗂️ Dicionário de dados automático (por arquivo)
- Para cada dataset encontrado, exibimos: dtype, % nulos, cardinalidade e amostra.

**1. deputados.json**

Descrição esperada: universo de parlamentares (id, nome, partido, UF, legislatura, contato).

In [4]:
df_deps = safe_read_json_records(FILES["deputados"])
if df_deps is not None:
    summary, sample = describe_dataframe(df_deps)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado.")


,column,dtype,pct_null,nunique
0,email,object,0.0,513
1,id,int64,0.0,513
2,idLegislatura,int64,0.0,1
3,nome,object,0.0,513
4,siglaPartido,object,0.0,20
5,siglaUf,object,0.0,27
6,uri,object,0.0,513
7,uriPartido,object,0.0,20
8,urlFoto,object,0.0,513


,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
1,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
2,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
3,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br
4,204528,https://dadosabertos.camara.leg.br/api/v2/depu...,Adriana Ventura,NOVO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adrianaventura@camara.leg.br


**2. eventos_sessoes.csv**

Descrição esperada: sessões deliberativas com id_evento, dataHoraInicio, dataHoraFim.

In [5]:
df_evt = safe_read_csv(FILES["eventos_sessoes"])
if df_evt is not None:
    summary, sample = describe_dataframe(df_evt)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado.")


[WARN] Falha ao ler CSV ../data/processed/eventos_sessoes.csv: No columns to parse from file
Arquivo não encontrado.


**3. freq_eventos.csv**

Descrição esperada: proxy de presença — contagem de eventos por deputado.

In [6]:
df_freq = safe_read_csv(FILES["freq_eventos"])
if df_freq is not None:
    summary, sample = describe_dataframe(df_freq)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado.")


,column,dtype,pct_null,nunique
0,id_deputado,int64,0.0,513
1,num_eventos,int64,0.0,439


,id_deputado,num_eventos
0,204379,908
1,220714,515
2,221328,441
3,204560,722
4,204528,1894


**4. votos_deputados.csv**

Descrição esperada: votos por votação (consolidados dos arquivos mensais).

In [7]:
df_votos = safe_read_csv(FILES["votos_deputados"])
if df_votos is not None:
    summary, sample = describe_dataframe(df_votos)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado.")


,column,dtype,pct_null,nunique
0,dataHoraRegistro,object,0.0,134889
1,id_deputado,int64,0.0,571
2,id_votacao,object,0.0,1190
3,tipo_voto,object,0.0,5


,id_votacao,id_deputado,tipo_voto,dataHoraRegistro
0,2236326-19,109429,Não,2020-02-05T21:55:05
1,2236326-19,204514,Não,2020-02-05T21:54:59
2,2236326-19,74160,Sim,2020-02-05T21:54:49
3,2236326-19,160642,Não,2020-02-05T21:54:39
4,2236326-19,92699,Sim,2020-02-05T21:54:27


**5. ocupacoes.csv**

Descrição esperada: ocupações vinculadas ao deputado (título, datas).

In [8]:
df_ocup = safe_read_csv(FILES["ocupacoes"])
if df_ocup is not None:
    summary, sample = describe_dataframe(df_ocup)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado.")


,column,dtype,pct_null,nunique
0,dataFim,float64,100.00,0
1,dataInicio,float64,100.00,0
2,id_deputado,int64,0.00,513
3,titulo,object,17.08,622


,id_deputado,titulo,dataInicio,dataFim
0,204379,NaN,NaN,NaN
1,220714,NaN,NaN,NaN
2,221328,NaN,NaN,NaN
3,204560,NaN,NaN,NaN
4,204528,Conselheira - Membro do Conselho Curador,NaN,NaN


**6. remuneracoes.csv**

Descrição esperada: proxy de status/condição via /deputados/{id} (situacao, condicaoEleitoral, data_inicio).

In [9]:
df_remu = safe_read_csv(FILES["remuneracoes"])
if df_remu is not None:
    summary, sample = describe_dataframe(df_remu)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado.")


,column,dtype,pct_null,nunique
0,cargo,object,0.0,3
1,data_inicio,object,0.0,81
2,id_deputado,int64,0.0,513
3,situacao,object,0.0,1


,id_deputado,cargo,situacao,data_inicio
0,204379,Titular,Exercício,2023-02-01
1,220714,Titular,Exercício,2023-02-01
2,221328,Suplente,Exercício,2024-03-21
3,204560,Titular,Exercício,2023-02-01
4,204528,Titular,Exercício,2023-02-01


**7. cod_situacao_deputados.json**

Descrição esperada: tabela de referência (siglas e nomes de situação).

df_sit = safe_read_json_records(FILES["cod_situacao_deputados"])
if df_sit is not None:
    summary, sample = describe_dataframe(df_sit)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado.")


**8. presencas.csv (em lab/data/processed)**

Descrição esperada: presenças por id_evento e id_deputado, com tipo_presenca.

In [10]:
df_pres = safe_read_csv(FILES["presencas"])
if df_pres is not None:
    summary, sample = describe_dataframe(df_pres)
    display(summary)
    display(sample)
else:
    print("Arquivo não encontrado (opcional, gerado no 01).")


,column,dtype,pct_null,nunique
0,ano_origem,int64,0.0,6
1,id_deputado,int64,0.0,924
2,id_evento,int64,0.0,10351
3,tipo_presenca,float64,100.0,0


,id_evento,id_deputado,tipo_presenca,ano_origem
0,58552,69871,NaN,2020
1,58552,73466,NaN,2020
2,58552,73696,NaN,2020
3,58552,74057,NaN,2020
4,58552,74200,NaN,2020


# 🔗 Relacionamentos e chaves

In [11]:
checks = []

# 1) PK de deputados
if df_deps is not None and "id" in df_deps.columns:
    checks.append(check_pk_unique(df_deps, ["id"]))

# 2) PK de eventos (se existir colunas típicas)
if df_evt is not None:
    evt_pk_cols = [c for c in df_evt.columns if c.lower() in {"id_evento","idevento"}]
    if evt_pk_cols:
        checks.append(check_pk_unique(df_evt, evt_pk_cols))

# 3) FK: freq_eventos.id_deputado -> deputados.id
if (df_freq is not None) and (df_deps is not None) and ("id_deputado" in df_freq.columns) and ("id" in df_deps.columns):
    checks.append(check_fk_in_pk(df_freq, ["id_deputado"], df_deps, ["id"]))

# 4) FK: presencas.id_evento -> eventos_sessoes.id_evento
if (df_pres is not None) and (df_evt is not None):
    # tenta mapear nome de coluna de evento
    evt_pk = [c for c in df_evt.columns if c.lower() in {"id_evento","idevento"}]
    if "id_evento" in df_pres.columns and evt_pk:
        checks.append(check_fk_in_pk(df_pres, ["id_evento"], df_evt, [evt_pk[0]]))

# 5) FK: votos_deputados.idDeputado (ou similar) -> deputados.id (nome pode variar conforme a API)
if (df_votos is not None) and (df_deps is not None) and ("id" in df_deps.columns):
    cand_cols = [c for c in df_votos.columns if c.lower() in {"iddeputado","id_deputado","idparlamentar","idecadastro"}]
    if cand_cols:
        checks.append(check_fk_in_pk(df_votos, [cand_cols[0]], df_deps, ["id"]))

# Consolida resultados
checks_df = pd.concat(checks, ignore_index=True) if checks else pd.DataFrame()
checks_df


,ok,msg,pk_cols,child_cols,parent_cols,missing
0,True,OK (sem duplicatas),[id],NaN,NaN,NaN
1,True,OK (todas as chaves filhas possuem pai),NaN,[id_deputado],[id],0.0
2,False,Falha: 208820 chaves filhas sem pai,NaN,[id_deputado],[id],208820.0


# 🔎 EDA rápida para “Compreensão dos Dados”


**Presença (proxy) — top/bottom por número de eventos**

In [12]:
if df_freq is not None:
    top10 = df_freq.sort_values("num_eventos", ascending=False).head(10).copy()
    bot10 = df_freq.sort_values("num_eventos", ascending=True).head(10).copy()

    if df_deps is not None and "id" in df_deps.columns and "nome" in df_deps.columns:
        top10 = top10.merge(df_deps[["id","nome"]], left_on="id_deputado", right_on="id", how="left")
        bot10 = bot10.merge(df_deps[["id","nome"]], left_on="id_deputado", right_on="id", how="left")

    display(top10[["id_deputado","nome","num_eventos"]])
    display(bot10[["id_deputado","nome","num_eventos"]])
else:
    print("freq_eventos.csv não encontrado.")


,id_deputado,nome,num_eventos
0,178993,Carlos Henrique Gaguim,4957
1,204539,Hercílio Coelho Diniz,2672
2,160575,Erika Kokay,2490
3,160598,Flávia Morais,2355
4,178929,Diego Garcia,2150
5,160518,Weliton Prado,2126
6,178871,Evair Vieira de Melo,2072
7,204412,Dr. Zacharias Calil,2032
8,204455,Luiz Lima,1965
9,204528,Adriana Ventura,1894


,id_deputado,nome,num_eventos
0,74065,Fatima Pelaes,11
1,233598,Samuel Santos,52
2,233594,Rafael Fera,54
3,141533,Rodrigo Rollemberg,79
4,233592,Paulo Lemos,86
5,230762,Pastor Claudio Mariano,90
6,230765,Enfermeira Rejane,153
7,231911,Lenir de Assis,160
8,230768,Zé Adriano,162
9,175765,Ribamar Silva,164


**Presenças — distribuição de tipo_presenca**

In [13]:
if df_pres is not None and "tipo_presenca" in df_pres.columns:
    dist = df_pres["tipo_presenca"].value_counts(dropna=False).to_frame("contagem")
    dist["pct"] = (dist["contagem"] / dist["contagem"].sum() * 100).round(2)
    dist
else:
    print("presencas.csv não encontrado ou sem coluna 'tipo_presenca'.")


**Votos — amostra por votação/deputado (checagem qualitativa)**

In [14]:
if df_votos is not None:
    cols_show = [c for c in df_votos.columns][:8]  # mostra primeiras colunas
    df_votos[cols_show].head(10)
else:
    print("votos_deputados.csv não encontrado.")


# Exportar “descrição_arquivos.csv”

In [15]:
catalog.to_csv(QUALITY_DIR / "descricao_arquivos.csv", index=False)
QUALITY_DIR / "descricao_arquivos.csv"


PosixPath('../data/quality/descricao_arquivos.csv')

# Checklist para responder a compreensão dos dados:
---

| Sub-seção                    | Onde responder no notebook              | O que usar                                                                                                                        |
| ---------------------------- | --------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------- |
| **Coleta dos dados**         | Markdown introdutório + Catálogo        | Tabela de *Fontes e Endpoints* do `01_coleta_dados` (copiar/ajustar), + comentários sobre paginação, retries e janelas temporais. |
| **Descrição dos dados**      | Blocos “Dicionário de dados automático” | `summary` (dtype, % nulos, cardinalidade) + `sample`.                                                                             |
| **Análise exploratória**     | EDA rápida                              | Top/bottom presença (freq_eventos), distribuição de tipo_presenca, amostras de votos.                                             |
| **Verificação de qualidade** | Relacionamentos e chaves                | Tabelas de PK/FK e mensagens de “OK”/“Falha”.                                                                                     |
